# Indhent wikipedia artikler

Formålet med denne notebook er at indhente Wikipedia-artikler der hører ind under minimum én af følgende tolv kategorier:

1.	UDANNELSE
2.	SAMFUND
3.	VIDENSKAB
4.	NATUR (KLIMA & MILJØ)
5.	TEKNOLOGI
6.	KULTUR (MUSIK, TEATER, KUNST)
7.	HISTORIE 
8.	SUNDHED
9.	GEOGRAFI
10.	ØKONOMI
11.	SPORT
12.	RELIGION
13. POLITIK
14. ERHVERVSLIV

Wikipedias artikler er inddelt i forskellige kategorier, som kan findes her: 

https://da.wikipedia.org/wiki/Wikipedia:Kategorier. 

Systemet er hierarkisk opbygget i en kringlet træ-struktur, hvor kategorier såsom *Historie*, *Sundhed*, *Geografi* m. fl. udgør topniveau-kategorier (https://da.wikipedia.org/wiki/Kategori:Topniveau_for_emner), som har en hel del underkategorier, der videre har en del underkategorier osv. Disse forgreninger af underkategorier kan sagtens være beslægtede - f.eks. hører Wikipedia artiklen "Arbejdsindsats" både ind under kategorierne *Sundhed* samt *Erhvervsliv*, som videre hører ind under *Økonomi* og *Samfund*. 

Kategorisystemet muliggør, at man kan indhente de Wikipedia-artikler, der hører ind under ovenstående tolv selvvalgte *hovedkategorier*. Dette gøres ved at starte med url-adresser for de tolv hovedkategorier. For hver hovedkategori, fx *Uddannelse*: "https://da.wikipedia.org/wiki/Kategori:Uddannelse", downloades først de tilhørende Wikipedia-artikler - for *Uddannelse* er det bl.a.artikler såsom Almendannelse, Elev, og Studiekort. Efterfølgende forfølges hvert link til underkategori-siderne, hvor de tilsvarende artikler ligeledes downloades osv. Da en artikel kan høre til flere kategorier vil denne måde at scrape på føre til artikel-gengangere. Disse vil vi håndtere i næste notebook.



# Indhent wikipedia url'er

Vi gemmer først url-adresserne for de tolv hovedkategorier i en liste

In [1]:
url_list = [
    "https://da.wikipedia.org/wiki/Kategori:Uddannelse",
    "https://da.wikipedia.org/wiki/Kategori:Samfund",
    "https://da.wikipedia.org/wiki/Kategori:Videnskab",
    "https://da.wikipedia.org/wiki/Kategori:Natur",
    "https://da.wikipedia.org/wiki/Kategori:Teknologi",
    "https://da.wikipedia.org/wiki/Kategori:Kultur",
    "https://da.wikipedia.org/wiki/Kategori:Historie",
    "https://da.wikipedia.org/wiki/Kategori:Sundhed",
    "https://da.wikipedia.org/wiki/Kategori:Geografi",
    "https://da.wikipedia.org/wiki/Kategori:%C3%98konomi",
    "https://da.wikipedia.org/wiki/Kategori:Sport",
    "https://da.wikipedia.org/wiki/Kategori:Religion",
    "https://da.wikipedia.org/wiki/Kategori:Politik",
    "https://da.wikipedia.org/wiki/Kategori:Erhvervsliv"
]

Herefter laver vi en funktion der vha. bibliotekerne *requests* og *bs4* givet en urladresse for en kategori scraper de tilhørende urladresser for hhv. underkategorier og artikler.

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
def get_urls(URL, type_):
    main_url = "https://da.wikipedia.org"
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    url_list = []
    if type_ == "subcategories":
        
        try:
            data = soup.findAll('div',attrs={'id':'mw-' + type_})
            for tag in data[0].find_all("a"):
                url_list.append(main_url + tag.attrs["href"])
            return url_list
        except:
            return []
    elif type_ == "pages":
        try:
            data = soup.findAll('div',attrs={'id':'mw-' + type_})
            for tag in data[0].find_all("a"):
                page_name = tag.attrs["href"]
                if not "Bruger:" in page_name and not "Portal:" in page_name:
                  url_list.append(main_url + page_name)
            return url_list
        except:
            return []

In [4]:
# Test funktion
print("Kategori:", url_list[6])
print(get_urls(url_list[6], "pages")[-3:])

Kategori: https://da.wikipedia.org/wiki/Kategori:Historie
['https://da.wikipedia.org/wiki/Liste_over_turistattraktioner_med_reenactment', 'https://da.wikipedia.org/wiki/Universalhistorie', 'https://da.wikipedia.org/wiki/Vestlig_historisk_t%C3%B8jmode']


For at gemme url-adresserne for hver hovedkategori benytter vi en dictionary med hovedkategorierne som keys. For hver key er den respektive value en liste med de tilhørende Wikipedia artikler. 

In [5]:
# Dictionary 
d = {"Uddannelse" : [],
    "Samfund": [],
    "Videnskab": [],
    "Natur": [],
    "Teknologi": [],
    "Kultur": [],
    "Historie": [],
    "Sundhed": [],
    "Geografi": [],
    "Økonomi": [],
    "Sport": [],
    "Religion": [],
    "Politik": [],
    "Erhvervsliv": []}
CATEGORIES = list(d.keys())

I det følgende looper vi gennem de tolv hovedkategorier. For hver hovedkategorier finder vi underkategorier ned til en dybde af 2 og scraper de respektive url-adresser for Wikipedia artiklerne.

In [6]:
%%time
DEPTH = 2 # Hvor mange undersider der scrapes
count = 1
for URL,category in zip(url_list,CATEGORIES):
    PAGE_URLS = []
    SUB_CATEGORIES = [[URL]]

    # get page urls - 1st layer
    for layer in range(DEPTH):
        s = []
        for url in SUB_CATEGORIES[layer]:
            PAGE_URLS.extend(get_urls(url, "pages"))
            s.extend(get_urls(url, "subcategories"))
        SUB_CATEGORIES.append(s)
        
    # store urls for the category
    d[category] = PAGE_URLS

    print(f"{count}/{len(CATEGORIES)}")
    count+=1

1/14
2/14
3/14
4/14
5/14
6/14
7/14
8/14
9/14
10/14
11/14
12/14
13/14
14/14
CPU times: total: 1min 21s
Wall time: 3min 14s


In [7]:
# Check størrelsen af dictionary
for cat in CATEGORIES:
  print(cat + ":", len(d[cat]))

Uddannelse: 598
Samfund: 2728
Videnskab: 915
Natur: 663
Teknologi: 312
Kultur: 2115
Historie: 1011
Sundhed: 517
Geografi: 372
Økonomi: 1183
Sport: 534
Religion: 1282
Politik: 1144
Erhvervsliv: 593


# Gem kategori-dictionary med url'er

In [8]:
import json

In [9]:
with open(f"urls-depth-{DEPTH}.json", "w") as f:
    json.dump(d, f)